In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_curve, auc
from matplotlib.legend_handler import HandlerLine2D
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_curve
import pandas as pd
import numpy as np

In [2]:
heros_full_features = pd.read_csv('dataset.csv')

# First: Separate the categorical from the boolean data


In [3]:
# Preprocessing
heros_nominal_information = heros_full_features.iloc[:, 2:11]
heros_boolean_information = heros_full_features.iloc[:, 11:]
heros_numerical_information = heros_full_features.loc[:, ['Height', 'Weight']]

In [4]:
# Remove Skin color, cause it is not good for the decision, then separete height and weight from the nominal atributes
heros_nominal_information = heros_nominal_information.drop(['Skin color', 'Height', 'Weight'], axis=1)

In [5]:
# Treat the missing numerical values giving then the mean of the others
imp = SimpleImputer(missing_values=-99, strategy='mean')
heros_numerical_information['Height'] = imp.fit_transform(np.array(heros_numerical_information['Height']).reshape(-1, 1))
heros_numerical_information['Weight'] = imp.fit_transform(np.array(heros_numerical_information['Weight']).reshape(-1, 1))

In [6]:
# Treat the missing nominal values giving then the most frequent of the others 
imp = SimpleImputer(missing_values='-', strategy='most_frequent')
# Labeling the nominal attributes 
le = preprocessing.LabelEncoder()
# All of this in the same loop

for index, item in heros_nominal_information.iteritems():
    heros_nominal_information.loc[:, index] = imp.fit_transform(np.array(heros_nominal_information.loc[:, index]).reshape(-1, 1))
    heros_nominal_information.loc[:, index] = le.fit_transform(heros_nominal_information.loc[:, index].astype(str))

In [7]:
# Discretize numerical attributes
#est = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
#heros_numerical_information = est.fit(heros_numerical_information)

In [8]:
pre_processed_dataset = pd.concat([heros_nominal_information, heros_boolean_information, heros_numerical_information], axis=1)

In [9]:
pre_processed_dataset.head()

,Gender,Eye color,Race,Hair color,Publisher,Alignment,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,...,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient,Height,Weight
0,1,18,22,14,11,1,False,True,False,False,...,False,False,False,False,False,False,False,False,203.000000,441.000000
1,1,2,31,14,2,1,True,True,False,False,...,False,False,False,False,False,False,False,False,191.000000,65.000000
2,1,2,53,14,1,1,False,False,True,False,...,False,False,False,False,False,False,False,False,185.000000,90.000000
3,1,7,26,14,11,0,False,True,False,False,...,False,False,False,False,False,False,False,False,203.000000,441.000000
4,1,2,10,1,11,0,False,False,False,True,...,False,False,False,False,False,False,False,False,187.237755,113.651709


In [10]:
# For Flight
target = pre_processed_dataset.loc[:, 'Alignment']
x_train, x_test, y_train, y_test = train_test_split(pre_processed_dataset.drop(['Alignment'], axis=1), target, test_size = 0.25, random_state = 42)

In [22]:
tree_classifier = DecisionTreeClassifier(random_state = 0, max_depth = 26)
tree_classifier.fit(x_train, y_train)

# Filling all the NaN occurences, with the set mean
print('Accuracy on the train subset: ' + str(tree_classifier.score(x_train, y_train)))
print('Accuracy on the test subset: ' + str(tree_classifier.score(x_test, y_test)))

Accuracy on the train subset: 0.9959595959595959
Accuracy on the test subset: 0.5393939393939394


In [16]:
depth = []
for i in range(3,33):
    clf = DecisionTreeClassifier(max_depth=i, random_state = 0)
    # Perform 7-fold cross validation 
    scores = cross_val_score(estimator=clf, X=pre_processed_dataset.drop(['Alignment'], axis=1), y=target, cv=7, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)

[(3, 0.6697005038333651), (4, 0.6697971765377755), (5, 0.6591752201368211), (6, 0.6392532678642385), (7, 0.6330448848614328), (8, 0.6136056618037887), (9, 0.6394633202773577), (10, 0.6350970511384845), (11, 0.6334002796111762), (12, 0.6122022949446367), (13, 0.6167769070927146), (14, 0.5999615330364346), (15, 0.6031767175316466), (16, 0.6015303035385305), (17, 0.6000888675016299), (18, 0.5788929147680457), (19, 0.5925710918511102), (20, 0.5790525988960046), (21, 0.5624128849061569), (22, 0.5804920245018862), (23, 0.557757941525055), (24, 0.5714531266395226), (25, 0.57589972701142), (26, 0.5819794424289962), (27, 0.5819471035172007), (28, 0.5758350491878288), (29, 0.5774034864099163), (30, 0.5789072458084126), (31, 0.5804110052069088), (32, 0.5804110052069088)]
